In [1]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np
from dateutil import parser
print 'mported files'

In [44]:
auc2011 = pd.read_csv('2011_auctions.csv')
pnames= []
players = auc2011['player_name']
for play in players:
    
    pnames.append(str(play.replace(' ','+')))


In [53]:

for player_name in pnames:
    print player_name 
    url = 'http://www.bing.com/search?q=cricketarchive.com+player+' + player_name
    print url
    #url  ='http://www.bing.com/search?q=cricketarchive.com+player+ms+dhoni'
    opener = urllib2.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    page = opener.open(url)
    soup = BeautifulSoup(page.read(),"html.parser")
    print 'bing sorted'
    #finding list a and t20 urls
    divs = soup.findAll('div',{'class':'b_attribution'})
    for div in divs:
        if str(div.text).find('cricketarchive.com/Archive/Players') != -1:

            print div.text
            s1 = div.text
            s2 = 'cricketarchive.com/Archive/Players'
            req_str =  s1[s1.index(s2) + len(s2):]
            wrds =  req_str.split('/')
            final_str = wrds[1]+ '/' + wrds[2]
            #print final_str
            listA_url = str('http://cricketarchive.com/Archive/Players/'+final_str+'/List_A_Matches.html')
            t20_url =   str('http://cricketarchive.com/Archive/Players/'+final_str+'/Twenty20_Matches.html')

            break
    #urls found out 
    
    listA_runs = []
    t20_runs = []
    listA_balls = []
    t20_balls = []
    listA_date = []
    t20_date = []
    
    
    page = opener.open(listA_url)
    soup = BeautifulSoup(page.read(),"html.parser")
    table = soup.findAll('table')
    
 
    rows =  table[0].findAll('tr')
    #print rows[0]
    date_start  = '2010-01-10'
    date_end  = '2011-01-10'

    for row in rows:
        tds  = row.findAll('td')
        #print tds[1].text
        date  = parser.parse(str(tds[1].text))
        if date > parser.parse(date_start) and date < parser.parse(date_end):
            print date.date()
            #listA_date.append(str(date.date()))
            mlink = tds[4].find('a')
            match_link = 'http://cricketarchive.com/' + mlink['href']
            print match_link
            match_page = opener.open(match_link)
            match_soup = BeautifulSoup(match_page.read(),"html.parser")
            #print match_soup

            found  = 0
            tabs = match_soup.findAll('table')


            rows = tabs[3].findAll('tr')
            for row in rows:
                tds  = row.findAll('td')
                if len(tds)  < 4:
                    continue
                if tds[0].find('a'):
                    link_text =  tds[0].find('a')['href']
                    if link_text.find(final_str) != -1:
                        #print tds[2].text
                        #print tds[3].text
                        listA_runs.append(tds[2].text)
                        listA_balls.append(tds[3].text)
                        found =  1
                        listA_date.append(str(date.date()))
                        break
            if (found== 1 ):
                continue


            rows = tabs[1].findAll('tr')
            for row in rows:
                tds  = row.findAll('td')
                if len(tds) < 4:
                    continue
                #print tds[0]
                if tds[0].find('a'):
                    link_text =  tds[0].find('a')['href']
                    if link_text.find(final_str) != -1:
                        listA_runs.append(tds[2].text)
                        listA_balls.append(tds[3].text)
                        listA_date.append(str(date.date()))
                        break
                        #print tds[2].text
                        #print tds[3].text      
    print 'work done for odis, now for t20s'
    
    page = opener.open(t20_url)
    soup = BeautifulSoup(page.read(),"html.parser")
    table = soup.findAll('table')
    rows =  table[0].findAll('tr')
    #print rows[0]
    date_start  = '2010-01-10'
    date_end  = '2011-01-10'

    for row in rows:
        tds  = row.findAll('td')
        #print tds[1].text
        date  = parser.parse(str(tds[1].text))
        if date > parser.parse(date_start) and date < parser.parse(date_end):
            print date.date()
            #t20_date.append(str(date.date()))
            mlink = tds[4].find('a')
            match_link = 'http://cricketarchive.com/' + mlink['href']
            print match_link
            match_page = opener.open(match_link)
            match_soup = BeautifulSoup(match_page.read(),"html.parser")
            #print match_soup

            found  = 0
            tabs = match_soup.findAll('table')


            rows = tabs[3].findAll('tr')
            for row in rows:
                tds  = row.findAll('td')
                if len(tds) < 4:
                    continue
                #print tds[0]
                if tds[0].find('a'):
                    link_text =  tds[0].find('a')['href']
                    if link_text.find(final_str) != -1:
                        #print tds[2].text
                        #print tds[3].text
                        t20_runs.append(tds[2].text)
                        t20_balls.append(tds[3].text)
                        found =  1
                        t20_date.append(str(date.date()))
                        break
            if (found== 1 ):
                continue


            rows = tabs[1].findAll('tr')
            for row in rows:
                tds  = row.findAll('td')
                #print tds[0]
                if len(tds)<4:
                    continue
                if tds[0].find('a'):
                    link_text =  tds[0].find('a')['href']
                    if link_text.find(final_str) != -1:
                        t20_runs.append(tds[2].text)
                        t20_balls.append(tds[3].text)
                        t20_date.append(str(date.date()))
                        break
                        #print tds[2].text
                        #print tds[3].text    
    print 't20s done'
    
    listA = pd.DataFrame()
    listA['date'] = listA_date
    listA['runs'] = listA_runs
    listA['balls'] = listA_balls
    fname = 'score_files/2011_listA_' + str(player_name) + '.csv'
    listA.to_csv(fname, encoding='utf-8')
    
    t20 = pd.DataFrame()
    t20['date'] = t20_date
    t20['runs'] = t20_runs
    t20['balls'] = t20_balls
    fname = 'score_files/2011_t20_' + str(player_name) + '.csv'
    t20.to_csv(fname, encoding='utf-8')



Gautam+Gambhir
http://www.bing.com/search?q=cricketarchive.com+player+Gautam+Gambhir
bing sorted
cricketarchive.com/Archive/Players/7/7736/7736.html
2010-01-11
http://cricketarchive.com//Archive/Scorecards/261/261636.html
2010-01-13
http://cricketarchive.com//Archive/Scorecards/261/261637.html
2010-06-16
http://cricketarchive.com//Archive/Scorecards/289/289653.html
2010-06-19
http://cricketarchive.com//Archive/Scorecards/289/289655.html
2010-06-22
http://cricketarchive.com//Archive/Scorecards/289/289657.html
2010-06-24
http://cricketarchive.com//Archive/Scorecards/289/289658.html
2010-11-28
http://cricketarchive.com//Archive/Scorecards/305/305597.html
2010-12-01
http://cricketarchive.com//Archive/Scorecards/305/305598.html
2010-12-04
http://cricketarchive.com//Archive/Scorecards/305/305599.html
2010-12-07
http://cricketarchive.com//Archive/Scorecards/305/305600.html
2010-12-10
http://cricketarchive.com//Archive/Scorecards/305/305601.html
work done for odis, now for t20s
2010-03-13
http

UnicodeEncodeError: 'ascii' codec can't encode character u'\xa0' in position 0: ordinal not in range(128)